In [1]:
import sklearn
import numpy as np
np.random.seed(42)
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

* 서포트 벡터 머신 

: 매우 강력하고 선형이나 비선형 분류, 회귀, 이상치 탐색에도 사용할 수 있는 다목적 머신러닝 모델이다. 복잡한 분류문제에 특히 잘 들어맞으며 작거나 중간 크기의 데이터셋에 적합하다.

# 5.1선형 SVM 분류 

* 라지마진분류

: 클래스를 잘 나누고, 제일 가까운 훈련 샘플로부터 가능한 한 멀리 떨어져 있다. 

: SVM 분류기를 클래스 사이에 가장 폭이 넓은 도로를 찾는 것으로 생각하기

* 서포트벡터

: 결정경계가 도로 경계에 위치한 샘플에 의해 전적으로 결정될 때 그 샘플 

### 5.1.1 소프트마진분류

* 하드마진분류

: 모든 샘플이 도로 바깥쪽에 올바르게 분류되어 있는 것 -> 문제점 : 데이터가 선형적으로 구분될 수 있어야 제대로 작동하고, 이상치에 민감하다. 

* 소프트 마진분류

: 하드마진분류의 문제를 피하기 위해 도로의 폭을 가능한 넓게 유지하고, 마진오류(즉 샘플이 도로 중간이나 반대쪽에 있는 경우) 사이에 적절한 균형을 잡은 분류

* C 하이퍼파라미터

: SVM 모델이 과대적합이라면 C를 감소시켜 모델을 규제할 수 있다. C를 높게 설정할수록 도로폭이 좁다. 

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

# 붓꽃 데이터셋을 적재하고 특성스케일을 변경하고 붓꽃의 품종을 감지하기 위해 선형 SVM 모델을 훈련시킨다. 

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]  # 꽃잎 길이, 꽃잎 너비
y = (iris["target"] == 2).astype(np.float64)  # Iris virginica

svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(C=1, loss="hinge", random_state=42)),
    ])

svm_clf.fit(X, y)

In [ ]:
svm_clf.predict([[5.5, 1.7]])

# 5.2 비선형 SVM 분류

비선형 데이터셋을 다루는 방법은 다항틑ㄱ성과 같은 특성을 더 추가하면 된다. 

In [ ]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

polynomial_svm_clf = Pipeline([
        ("poly_features", PolynomialFeatures(degree=3)),
        ("scaler", StandardScaler()),
        ("svm_clf", LinearSVC(C=10, loss="hinge", random_state=42))
    ])

polynomial_svm_clf.fit(X, y)

### 5.2.1 다항식 커널

* 다항식 특성을 추가하는 것은 간단하고 모든 머신러닝 알고리즘에서 잘 돌아간다. 그러나 낮은 차수의 다항식은 매우 복잡한 데이터셋을 잘 표현하지 못하고 높은 차수의 다항식은 굉장히 많은 특성을 추가하므로 모델을 느리게 만든다.

* 다행히도 SVM을 사용할 땐, kernel trick (커널트릭) 이라는 수학적 기교를 적용할 수 있다. 커널트릭은 실제로는 특성을 추가하지 않으면서 다항식 특성을 많이 추가한 것과 같은 결과를 얻을 수 있다. 

In [ ]:
from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
    ])
poly_kernel_svm_clf.fit(X, y)

### 5.2.2 유사도 특성

* 비선형 특성을 다루는 또다른 기법으로는 각 샘플이 특정 '랜드마크'와 얼마나 닮았는지 측정하는 유사도 함수로 계산한 특성을 추가하는 것이다. 

### 5.2.3 가우시간 RBF 커널 

* 가우시간 RBF 함수의 값은 0(랜드마크에서 아주 멀리 떨어져있는 경우)부터 1(랜드마크와 같은 위치인 경우) 까지 변화하며 종 모양으로 나타낸다. 

* 랜드마크 설정 방법 : 가장 간단한 방법은 데이터셋에 있는 모든 샘플 위치에 랜드마크를 설정하는 것. 이렇게 하면 차원이 매우 커지고 변환된 훈련 세트가 선형적으로 구분될 가능성이 높다. 

In [ ]:
rbf_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
    ])
rbf_kernel_svm_clf.fit(X, y)

In [ ]:
from sklearn.svm import SVC

gamma1, gamma2 = 0.1, 5
C1, C2 = 0.001, 1000
hyperparams = (gamma1, C1), (gamma1, C2), (gamma2, C1), (gamma2, C2)

svm_clfs = []
for gamma, C in hyperparams:
    rbf_kernel_svm_clf = Pipeline([
            ("scaler", StandardScaler()),
            ("svm_clf", SVC(kernel="rbf", gamma=gamma, C=C))
        ])
    rbf_kernel_svm_clf.fit(X, y)
    svm_clfs.append(rbf_kernel_svm_clf)

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10.5, 7), sharex=True, sharey=True)

for i, svm_clf in enumerate(svm_clfs):
    plt.sca(axes[i // 2, i % 2])
    plot_predictions(svm_clf, [-1.5, 2.45, -1, 1.5])
    plot_dataset(X, y, [-1.5, 2.45, -1, 1.5])
    gamma, C = hyperparams[i]
    plt.title(r"$\gamma = {}, C = {}$".format(gamma, C), fontsize=16)
    if i in (0, 1):
        plt.xlabel("")
    if i in (1, 3):
        plt.ylabel("")
plt.show()

### 5.2.4 계산복잡도

* 선형 SVM의 구현 liblinear 라이브러리의 알고리즘의 훈련시간 복잡도 : O(mxn)
  - 정밀도를 높이면 알고리즘의 수행시간이 길어진다.
  - SVM은 커널트릭알고리즘을 구현한 libsvm 라이브러리를 기반한다. 

# 5.3 SVM 회귀

* SVM은 선형, 비선형 분류 뿐만 아니라 선형, 비선형 회귀에도 사용될 수 있다. 

* 일정한 마진 오류 안에서 두 클래스 간의 도로폭이 가능한 최대로 하는 대신, SVM 회귀는 제한된 마진오류 (도로 밖의 샘플) 안에서 도로 안에 가능한 한 많은 샘플이 들어가도록 학습한다. 

* C 파라미터 : C가 크다(=규제가 거의 없다), C가 작다(=규제가 훨씬 많다)

In [ ]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5, random_state=42)
svm_reg.fit(X, y)

In [ ]:
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1, gamma="scale")
svm_poly_reg.fit(X, y)